<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/prepare_data_momentum_algo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from statsmodels.regression.rolling import RollingOLS
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_nifty50_tickers():
    #data = pd.read_csv('https://raw.githubusercontent.com/anirbanghoshsbi/data/main/STOCKS100.txt', sep='\t')
    nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
    nifty50 = pd.read_csv(nifty50_url)
    tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
    #reject_lst=['MOTHERSUMI.NS', 'CADILAHC.NS', 'ADANITRANS.NS', 'SRTRANSFIN.NS', 'INFRATEL-EQ.NS','HDFC.NS','LTI.NS']
    #nifty_final= [item for item in nifty50 if item not in reject_lst]

    return tickers


In [4]:
symbols_list = get_nifty50_tickers()

end_date = '2023-09-27'

start_date = '2021-01-02'
df = yf.download(tickers=symbols_list,
                 start=start_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()



[*********************100%%**********************]  50 of 50 completed


In [5]:
df

Price                        adj close         close          high  \
date       ticker                                                    
2021-01-04 ADANIENT.NS      493.269409    494.500000    502.600006   
           ADANIPORTS.NS    486.647064    498.899994    510.899994   
           APOLLOHOSP.NS   2383.384766   2409.250000   2429.000000   
           ASIANPAINT.NS   2668.798828   2753.699951   2791.100098   
           AXISBANK.NS      621.989929    624.700012    633.000000   
...                                ...           ...           ...   
2024-08-16 TECHM.NS        1585.300049   1585.300049   1589.000000   
           TITAN.NS        3444.750000   3444.750000   3450.100098   
           ULTRACEMCO.NS  11295.700195  11295.700195  11325.000000   
           UPL.NS           553.349976    553.349976    554.400024   
           WIPRO.NS         516.250000    516.250000    517.450012   

Price                              low          open    volume  
date       ticker                                               
2021-01-04 ADANIENT.NS      486.649994    492.000000   4936582  
           ADANIPORTS.NS    495.200012    507.000000   9998803  
           APOLLOHOSP.NS   2398.250000   2418.100098    527018  
           ASIANPAINT.NS   2745.000000   2781.100098   1360486  
           AXISBANK.NS      621.299988    627.799988  14068156  
...                                ...           ...       ...  
2024-08-16 TECHM.NS        1531.300049   1558.400024   3224481  
           TITAN.NS        3370.199951   3410.000000   1161982  
           ULTRACEMCO.NS  10950.200195  11040.000000    435631  
           UPL.NS           539.599976    542.849976   1635134  
           WIPRO.NS         497.950012    501.000000  12965570  

[44650 rows x 6 columns]

In [6]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                        adj close         close          high  \
date       ticker                                                    
2021-01-04 ADANIENT.NS      493.269409    494.500000    502.600006   
           ADANIPORTS.NS    486.647064    498.899994    510.899994   
           APOLLOHOSP.NS   2383.384766   2409.250000   2429.000000   
           ASIANPAINT.NS   2668.798828   2753.699951   2791.100098   
           AXISBANK.NS      621.989929    624.700012    633.000000   
...                                ...           ...           ...   
2024-08-16 TECHM.NS        1585.300049   1585.300049   1589.000000   
           TITAN.NS        3444.750000   3444.750000   3450.100098   
           ULTRACEMCO.NS  11295.700195  11295.700195  11325.000000   
           UPL.NS           553.349976    553.349976    554.400024   
           WIPRO.NS         516.250000    516.250000    517.450012   

Price                              low          open    volume  \
date       ticker                                                
2021-01-04 ADANIENT.NS      486.649994    492.000000   4936582   
           ADANIPORTS.NS    495.200012    507.000000   9998803   
           APOLLOHOSP.NS   2398.250000   2418.100098    527018   
           ASIANPAINT.NS   2745.000000   2781.100098   1360486   
           AXISBANK.NS      621.299988    627.799988  14068156   
...                                ...           ...       ...   
2024-08-16 TECHM.NS        1531.300049   1558.400024   3224481   
           TITAN.NS        3370.199951   3410.000000   1161982   
           ULTRACEMCO.NS  10950.200195  11040.000000    435631   
           UPL.NS           539.599976    542.849976   1635134   
           WIPRO.NS         497.950012    501.000000  12965570   

Price                     garman_klass_vol        rsi    bb_low    bb_mid  \
date       ticker                                                           
2021-01-04 ADANIENT.NS        5.174493e-04        NaN       NaN       NaN   
           ADANIPORTS.NS     -1.613715e-04        NaN       NaN       NaN   
           APOLLOHOSP.NS      3.817025e-07        NaN       NaN       NaN   
           ASIANPAINT.NS     -5.175967e-04        NaN       NaN       NaN   
           AXISBANK.NS        1.406361e-04        NaN       NaN       NaN   
...                                    ...        ...       ...       ...   
2024-08-16 TECHM.NS           5.709073e-04  65.977583  7.281409  7.322527   
           TITAN.NS           2.347987e-04  54.764107  8.085322  8.129303   
           ULTRACEMCO.NS      3.638019e-04  50.031397  9.303603  9.347530   
           UPL.NS             2.243024e-04  51.678196  6.262813  6.305662   
           WIPRO.NS           3.904698e-04  53.239825  6.160340  6.228743   

Price                      bb_high       atr      macd  dollar_volume  
date       ticker                                                      
2021-01-04 ADANIENT.NS         NaN       NaN       NaN    2435.064887  
           ADANIPORTS.NS       NaN       NaN       NaN    4865.888126  
           APOLLOHOSP.NS       NaN       NaN       NaN    1256.086672  
           ASIANPAINT.NS       NaN       NaN       NaN    3630.863442  
           AXISBANK.NS         NaN       NaN       NaN    8750.251354  
...                            ...       ...       ...            ...  
2024-08-16 TECHM.NS       7.363645  1.100010  0.738285    5111.769887  
           TITAN.NS       8.173284  1.337339 -0.155071    4002.737495  
           ULTRACEMCO.NS  9.391456  2.483500 -0.633752    4920.757172  
           UPL.NS         6.348510 -0.181049 -0.046903     904.801359  
           WIPRO.NS       6.297145  0.521918 -0.599512    6693.475513  

[44650 rows x 14 columns]

In [7]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('W').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('W').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume     adj close  garman_klass_vol  \
date       ticker                                                         
2021-02-14 ADANIENT.NS      5128.580313    717.160889          0.002082   
           ADANIPORTS.NS    8723.193078    601.456238          0.001643   
           APOLLOHOSP.NS    1986.904736   2718.597168          0.000143   
           ASIANPAINT.NS    8748.005250   2409.449463         -0.000472   
           AXISBANK.NS     11904.590334    747.144653          0.000271   
...                                 ...           ...               ...   
2024-08-18 TECHM.NS         3260.584815   1585.300049          0.000571   
           TITAN.NS         3687.265447   3444.750000          0.000235   
           ULTRACEMCO.NS    3444.910000  11295.700195          0.000364   
           UPL.NS           1051.888137    553.349976          0.000224   
           WIPRO.NS         3195.965707    516.250000          0.000390   

                                rsi    bb_low    bb_mid   bb_high       atr  \
date       ticker                                                             
2021-02-14 ADANIENT.NS    86.404783  6.136371  6.336293  6.536214 -1.343859   
           ADANIPORTS.NS  74.374523  6.185985  6.288131  6.390277 -0.448093   
           APOLLOHOSP.NS  63.929038  7.830363  7.881283  7.932204 -1.228014   
           ASIANPAINT.NS  40.716782  7.720908  7.796483  7.872057  0.952396   
           AXISBANK.NS    65.886314  6.434630  6.542819  6.651007  1.566797   
...                             ...       ...       ...       ...       ...   
2024-08-18 TECHM.NS       65.977583  7.281409  7.322527  7.363645  1.100010   
           TITAN.NS       54.764107  8.085322  8.129303  8.173284  1.337339   
           ULTRACEMCO.NS  50.031397  9.303603  9.347530  9.391456  2.483500   
           UPL.NS         51.678196  6.262813  6.305662  6.348510 -0.181049   
           WIPRO.NS       53.239825  6.160340  6.228743  6.297145  0.521918   

                              macd  
date       ticker                   
2021-02-14 ADANIENT.NS    0.270671  
           ADANIPORTS.NS  0.773970  
           APOLLOHOSP.NS  0.479209  
           ASIANPAINT.NS -1.848625  
           AXISBANK.NS    1.732751  
...                            ...  
2024-08-18 TECHM.NS       0.738285  
           TITAN.NS      -0.155071  
           ULTRACEMCO.NS -0.633752  
           UPL.NS        -0.046903  
           WIPRO.NS      -0.599512  

[9200 rows x 9 columns]

In [8]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df


data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume     adj close  garman_klass_vol  \
date       ticker                                                         
2021-05-09 ADANIENT.NS     10175.487626   1295.667725          0.000279   
           ADANIPORTS.NS   17638.087114    748.894165          0.000924   
           APOLLOHOSP.NS    3188.061580   3275.305664          0.000220   
           ASIANPAINT.NS    3106.315470   2472.978027         -0.000233   
           AXISBANK.NS      7558.309134    713.640625          0.000120   
...                                 ...           ...               ...   
2024-08-18 TECHM.NS         3260.584815   1585.300049          0.000571   
           TITAN.NS         3687.265447   3444.750000          0.000235   
           ULTRACEMCO.NS    3444.910000  11295.700195          0.000364   
           UPL.NS           1051.888137    553.349976          0.000224   
           WIPRO.NS         3195.965707    516.250000          0.000390   

                                rsi    bb_low    bb_mid   bb_high       atr  \
date       ticker                                                             
2021-05-09 ADANIENT.NS    71.319270  6.974810  7.074190  7.173569 -0.658390   
           ADANIPORTS.NS  56.033375  6.524793  6.596742  6.668692  0.727615   
           APOLLOHOSP.NS  62.275343  7.987391  8.052386  8.117381 -0.656816   
           ASIANPAINT.NS  50.734780  7.793853  7.824410  7.854967  0.099865   
           AXISBANK.NS    54.216680  6.449666  6.526504  6.603343  0.411660   
...                             ...       ...       ...       ...       ...   
2024-08-18 TECHM.NS       65.977583  7.281409  7.322527  7.363645  1.100010   
           TITAN.NS       54.764107  8.085322  8.129303  8.173284  1.337339   
           ULTRACEMCO.NS  50.031397  9.303603  9.347530  9.391456  2.483500   
           UPL.NS         51.678196  6.262813  6.305662  6.348510 -0.181049   
           WIPRO.NS       53.239825  6.160340  6.228743  6.297145  0.521918   

                              macd  return_1m  return_2m  return_3m  \
date       ticker                                                     
2021-05-09 ADANIENT.NS    0.463828   0.120998   0.059275   0.038717   
           ADANIPORTS.NS -0.048334   0.051640   0.028812   0.005376   
           APOLLOHOSP.NS  0.525470   0.035450   0.017165   0.026499   
           ASIANPAINT.NS  0.185813   0.006012   0.006670  -0.014223   
           AXISBANK.NS    0.013546   0.002588   0.033259   0.023145   
...                            ...        ...        ...        ...   
2024-08-18 TECHM.NS       0.738285   0.052167   0.025412   0.009459   
           TITAN.NS      -0.155071   0.033932  -0.002545  -0.004735   
           ULTRACEMCO.NS -0.633752  -0.000411  -0.020127  -0.008942   
           UPL.NS        -0.046903  -0.000452   0.015459   0.006209   
           WIPRO.NS      -0.599512   0.050784   0.013942  -0.005460   

                          return_6m  return_9m  return_12m  
date       ticker                                           
2021-05-09 ADANIENT.NS     0.046601   0.040341    0.050525  
           ADANIPORTS.NS   0.014781   0.002758    0.018438  
           APOLLOHOSP.NS   0.025194   0.015631    0.015646  
           ASIANPAINT.NS   0.003070   0.007434    0.002171  
           AXISBANK.NS     0.004380  -0.002147   -0.003816  
...                             ...        ...         ...  
2024-08-18 TECHM.NS        0.016767   0.018306    0.016821  
           TITAN.NS        0.008748  -0.002352    0.001059  
           ULTRACEMCO.NS  -0.004724   0.001179    0.008780  
           UPL.NS         -0.005413   0.000532    0.006024  
           WIPRO.NS       -0.005959   0.008707    0.008995  

[8600 rows x 15 columns]

In [9]:
data.to_csv('nifty_50_weekly_data.csv')